# Using LightFM for Recommendations

Check out LightFM here https://lyst.github.io/lightfm/docs/index.html


In [0]:
!pip install lightfm

     |████████████████████████████████| 307kB 5.1MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707625 sha256=ae726a1ca1e739b8a7632d51900932bc7552d53bf11f51568d81dc362b3a6d50
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


The first step is to get the Movielens data. This is a classic small recommender dataset, consisting of around 950 users, 1700 movies, and 100,000 ratings. The ratings are on a scale from 1 to 5, but we’ll all treat them as implicit positive feedback in this example.

Fortunately, this is one of the functions provided by LightFM itself.

In [0]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

In [0]:
data = fetch_movielens(min_rating = 4.0)

for key, value in data.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [0]:
type(data['train'])

scipy.sparse.coo.coo_matrix

In [0]:
# Each row represents a user, and each column an item. Entries are ratings from 1 to 5.
m1 = data['train'].tocsr()

print(m1[0,0])
print(m1[0,1])

5
0


**coo_matrix - Intended Usage**

- COO is a fast format for constructing sparse matrices
- Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector operations
- By default when converting to CSR or CSC format, duplicate (i,j) entries will be summed together.  This facilitates efficient construction of finite element matrices and the like. (see example)

In [0]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


# Let's now create and train our model

In [0]:
model = LightFM(loss = 'warp')

In [0]:
train = data['train']
test = data['test']

model.fit(train, epochs=10)

In [0]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.49, test 0.08.
AUC: train 0.94, test 0.91.


# Let's see what movies are recommended for some users

In [0]:
# Function credit goes to Arun Mathew Kurian
# https://towardsdatascience.com/how-to-build-a-movie-recommender-system-in-python-using-lightfm-8fa49d7cbe3b
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data['train'].shape
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))

        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("Known positives:")
        
        for x in known_positives[:3]:
            print("%s" % x)
        
        print("Recommended:")
        for x in top_items[:3]:
            print("%s" % x)

In [0]:
# Testing on users 6, 125 and 336
sample_recommendation(model, data, [6, 125, 336])

User 6
Known positives:
Get Shorty (1995)
Twelve Monkeys (1995)
Babe (1995)
Recommended:
Raiders of the Lost Ark (1981)
Princess Bride, The (1987)
Empire Strikes Back, The (1980)
User 125
Known positives:
Jungle2Jungle (1997)
Kull the Conqueror (1997)
Scream (1996)
Recommended:
Air Force One (1997)
Game, The (1997)
Kiss the Girls (1997)
User 336
Known positives:
Mr. Holland's Opus (1995)
Star Wars (1977)
Ace Ventura: Pet Detective (1994)
Recommended:
Star Wars (1977)
Independence Day (ID4) (1996)
Raiders of the Lost Ark (1981)


### Learn to build and create your own datasets for LightFM here

https://lyst.github.io/lightfm/docs/examples/dataset.html